# АНАЛИЗ ПОЛЬЗОВАТЕЛЬСКИХ СОБЫТИЙ

In [144]:
# Таблица Event = events_2018
# id - идентификатор события
# user_id - уникальный идентификатор пользователя, совершившего событие в приложении
# start_time - дата и время события
# event_type - тип события (значения: registration — регистрация; tutorial_start — начало обучения; tutorial_finish — завершение обучения;/ 
# /level_choice — выбор уровня сложности; pack_choice — выбор пакетов вопросов)
# tutorial_id	идентификатор обучения (этот идентификатор есть только у событий обучения)
# selected_level	выбранный уровень сложности обучения

# Таблица purchase = purchase_2018
# id - идентификатор события
# user_id - уникальный идентификатор пользователя, совершившего событие в приложении
# event_datetime - дата и время события/покупки
# amount - сумма оплаты

импортируем готовые датафреймы(таблицы) из предыдущего файла:

In [145]:
import pandas as pd
import import_ipynb
from Game_Quiz_Freeze1_1 import purchase_2018, events_2018

display(events_2018.head())
display(purchase_2018.head())

,event_id,event_type,selected_level,start_time,tutorial_id,user_id
51405,80308,registration,NaN,2018-01-01 03:48:40,NaN,27832
51406,80309,registration,NaN,2018-01-01 04:07:25,NaN,27833
51407,80310,registration,NaN,2018-01-01 08:35:10,NaN,27834
51408,80311,registration,NaN,2018-01-01 11:54:47,NaN,27835
51409,80312,registration,NaN,2018-01-01 13:28:07,NaN,27836


,purchase_id,user_id,start_time,amount,event_type
1171,16845,27845,2018-01-03 18:53:43,100,purchase
1172,16846,27865,2018-01-04 14:46:10,250,purchase
1174,16848,27911,2018-01-07 08:19:12,50,purchase
1175,16849,27910,2018-01-07 12:11:34,100,purchase
1176,16850,27940,2018-01-07 13:16:41,200,purchase


In [146]:
purchase_2018.describe()

,purchase_id,user_id,amount
count,1600.000000,1600.000000,1600.000000
mean,17645.505625,37752.766250,110.734375
std,462.038637,5822.621784,54.696628
min,16845.000000,27845.000000,25.000000
25%,17245.750000,32815.750000,50.000000
50%,17645.500000,37633.500000,100.000000
75%,18045.250000,43023.000000,150.000000
max,18452.000000,47742.000000,300.000000


# Исследуем пропуски и более детально знакомимся с датасетами


In [147]:
events_2018[events_2018['event_type'] == 'level_choice'].info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 8342 entries, 51424 to 118363
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   event_id        8342 non-null   int64         
 1   event_type      8342 non-null   object        
 2   selected_level  8342 non-null   object        
 3   start_time      8342 non-null   datetime64[ns]
 4   tutorial_id     0 non-null      float64       
 5   user_id         8342 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 456.2+ KB


Как видно, этот срез датафрейма не содержит пропущенных значений в столбце selected_level, но зато содержит пропуски в tutorial_id. Это связано с тем, что для событий типа level_choice не предусмотрена запись параметра tutorial_id.

In [148]:
events_2018[events_2018['event_type'].isin(['tutorial_start','tutorial_finish'])].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32954 entries, 51412 to 118360
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   event_id        32954 non-null  int64         
 1   event_type      32954 non-null  object        
 2   selected_level  0 non-null      object        
 3   start_time      32954 non-null  datetime64[ns]
 4   tutorial_id     32954 non-null  float64       
 5   user_id         32954 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 1.8+ MB


Столбец selected_level в таком срезе не содержит ни одного значения. Причина этого в том, что для событий, связанных с обучением, отсутствуют параметры выбора уровня сложности selected_level. А вот остальные столбцы заполнены полностью.

In [149]:
# возможные события в event_type и selected_level
print(events_2018['event_type'].unique(), ' - event_type')
print(events_2018['selected_level'].unique(), ' - selected_level')

['registration' 'tutorial_start' 'tutorial_finish' 'level_choice'
 'pack_choice']  - event_type
[nan 'medium' 'hard' 'easy']  - selected_level


In [150]:
print(events_2018[events_2018['event_type'] == 'registration']['user_id'].nunique(), " - количество регистраций")
print(events_2018['user_id'].nunique(), ' - количество user_id')
print(events_2018[events_2018['event_type'] == 'tutorial_start']['user_id'].nunique(), " - начали обучение")
print(events_2018[events_2018['event_type'] == 'tutorial_finish']['user_id'].nunique(), " - закончили обучение")
print(events_2018[events_2018['event_type'] == 'level_choice']['user_id'].nunique(), " - выбрали уровень сложности")
print(events_2018[events_2018['event_type'] == 'pack_choice']['user_id'].nunique(), ' - выбрали бесплатный пакет вопросов')
print(purchase_2018['user_id'].nunique(), ' - количество оплат')


19926  - количество регистраций
19926  - количество user_id
11858  - начали обучение
10250  - закончили обучение
8342  - выбрали уровень сложности
5737  - выбрали бесплатный пакет вопросов
1600  - количество оплат


регистрация обязательный параметр

In [151]:
registered_users_count = events_2018[events_2018["event_type"] == "registration"][
    "user_id"
].nunique()
tutorial_start_users_count = events_2018[events_2018["event_type"] == "tutorial_start"][
    "user_id"
].nunique()
percent_tutorial_start_users = tutorial_start_users_count / registered_users_count
print(
    "Процент пользователей, начавших обучение (от общего числа зарегистрировавшихся): {:.2%}".format(
        percent_tutorial_start_users
    )
)

Процент пользователей, начавших обучение (от общего числа зарегистрировавшихся): 59.51%


In [152]:
tutorial_finish_users_count = events_2018[events_2018["event_type"] == "tutorial_finish"][
    "user_id"
].nunique()
tutorial_completion_rate = tutorial_finish_users_count / tutorial_start_users_count
print(
    "Процент пользователей, завершивших обучение: {:.2%}".format(
        tutorial_completion_rate
    )
)

Процент пользователей, завершивших обучение: 86.44%


In [153]:
level_choice_users_count = events_2018[events_2018["event_type"] == "level_choice"][
    "user_id"
].nunique()
percent_level_choice_users = level_choice_users_count / registered_users_count
print(
    "Процент пользователей, выбравших уровень сложности (от общего числа зарегистрировавшихся): {:.2%}".format(
        percent_level_choice_users
    )
)

Процент пользователей, выбравших уровень сложности (от общего числа зарегистрировавшихся): 41.86%


In [154]:
training_choice_users_count = events_2018[events_2018["event_type"] == "pack_choice"][
    "user_id"
].nunique()
percent_training_choice_users = training_choice_users_count / level_choice_users_count
print(
    "Процент пользователей, выбравших пакет бесплатных вопросов (от числа пользователей, которые выбрали уровень сложности): {:.2%}".format(
        percent_training_choice_users
    )
)

Процент пользователей, выбравших пакет бесплатных вопросов (от числа пользователей, которые выбрали уровень сложности): 68.77%


In [155]:
paying_users_count = purchase_2018["user_id"].nunique()
percent_of_paying_users = paying_users_count / training_choice_users_count
print(
    "Процент пользователей, которые совершили оплату (от числа пользователей, которые воспользовались бесплатными возможностями): {:.2%}".format(
        percent_of_paying_users
    )
)

Процент пользователей, которые совершили оплату (от числа пользователей, которые воспользовались бесплатными возможностями): 27.89%


In [156]:
purchase_rate = paying_users_count / registered_users_count
print(
    "Процент пользователей, которые оплатили набор вопросов (от числа зарегистрировавшихся пользователей): {:.2%}".format(
        purchase_rate
    )
)

Процент пользователей, которые оплатили набор вопросов (от числа зарегистрировавшихся пользователей): 8.03%
